In [2]:
#import the relevant libraries
import requests
import json

#load the keys and instantiate the search terms
def get_keys(path):
    with open(path) as f:
        return json.load(f)

keys = get_keys("C:/Users/GM/.secret/yelp_api.json")

api_key = keys['api_key']
api_host = "https://api.yelp.com/v3"

term = "restaurants"
location = "New York City"

headers = {
    "Authorization": f"Bearer {api_key}"
}

params = {
    "term": term,
    "location": location,
    "limit": 50,
    "offset": 10 #start from the first page
}

restaurants = []

response = requests.get(f"{api_host}/businesses/search", headers=headers, params=params)

print(f'the keys available {json.loads(response.text).keys()}')
total_results = json.loads(response.text)["total"]

while len(restaurants) < min(total_results, 500):
    search_results = json.loads(response.text)
    restaurant_list = search_results["businesses"]

    for restaurant in restaurant_list:
        name = restaurant["name"]
        rating = restaurant["rating"]
        pricing = restaurant.get("price")
        url = restaurant["url"]
        latitude = restaurant["coordinates"]["latitude"]
        longitude = restaurant["coordinates"]["longitude"]
        transactions = restaurant.get("transactions")
        location = ", ".join(restaurant["location"]["display_address"])
        phone = restaurant["phone"]
        display_phone = restaurant["display_phone"]
        distance = restaurant.get("distance")
        cuisine = restaurant["categories"][0]["title"] # Extract the type of cuisine
        resturant_id = restaurant["id"] # Extract the restaurant ID
        print(f'name of restaurant {name}')
        
        reviews = []
        reviews_endpoint = f"{api_host}/businesses/{restaurant['id']}/reviews"
        reviews_response = requests.get(reviews_endpoint, headers=headers)
        reviews_data = json.loads(reviews_response.text)
        # print(reviews_data.keys())
        for review_data in reviews_data["reviews"]:
            review_text = review_data["text"]
            reviewer_name = review_data["user"]["name"]
            reviewer_id = review_data["user"]["id"]
            rating = review_data["rating"]
            reviews.append({
                "text": review_text,
                "reviewer": reviewer_name,
                "reviewer_id": reviewer_id,
                "rating": rating
            })
        
        print(f'length of reviews for {name} is {len(reviews)}')
        
        restaurants.append({
            "restaurant_id": resturant_id, # Add the restaurant ID to restaurant data
            "name": name,
            "rating": rating,
            "pricing": pricing,
            "url": url,
            "latitude": latitude,
            "longitude": longitude,
            "transactions": transactions,
            "location": location,
            "phone": phone,
            "display_phone": display_phone,
            "distance": distance,
            "reviews": reviews,
            "review_count": len(reviews),
            "cuisine": cuisine, # Add the type of cuisine to restaurant data           
            "num_reviews": restaurant["review_count"] # Add the number of reviews to restaurant data
        })

    if len(restaurants) >= min(total_results, 500) or (params["offset"] + 50) >= total_results:
        break

    params["offset"] += 50
    response = requests.get(f"{api_host}/businesses/search", headers=headers, params=params)


the keys available dict_keys(['error'])


KeyError: 'total'

In [ ]:
import csv

# Code for retrieving restaurant data

# Save restaurant data to CSV file
with open('restaurant_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Restaurant ID', 'Name', 'Rating', 'Pricing', 'URL', 'Latitude', 'Longitude', 'Transactions', 'Location', 'Phone', 'Display Phone', 'Distance', 'Reviews', 'Review Count', 'Cuisine',  'Number of Reviews'])
    for restaurant in restaurants:
        writer.writerow(restaurant['restaurant_id'],[restaurant['name'], restaurant['rating'], restaurant['pricing'], restaurant['url'], restaurant['latitude'], restaurant['longitude'], restaurant['transactions'], restaurant['location'], restaurant['phone'], restaurant['display_phone'], restaurant['distance'], restaurant['reviews'], restaurant['review_count'], restaurant['cuisine'], restaurant['num_reviews']])
print('Restaurant data saved as CSV file.')